In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Optional
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ==================== 1. БАЗОВЫЕ КОМПОНЕНТЫ ====================

class Linear(nn.Module):
    """
    Кастомная реализация линейного слоя
    """
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Инициализация весов
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        
        # Инициализация смещения
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
    
    def reset_parameters(self):
        """Инициализация параметров по методу Xavier/Glorot"""
        # Более стабильная инициализация
        nn.init.xavier_uniform_(self.weight, gain=1.0)
        if self.bias is not None:
            nn.init.zeros_(self.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Прямой проход"""
        # Поддержка многомерных тензоров
        if x.dim() > 2:
            # Сохраняем форму
            original_shape = x.shape
            # Выпрямляем все кроме последнего измерения
            x = x.reshape(-1, original_shape[-1])
            output = torch.matmul(x, self.weight.t())
            if self.bias is not None:
                output += self.bias
            # Восстанавливаем форму
            output = output.reshape(*original_shape[:-1], self.out_features)
        else:
            output = torch.matmul(x, self.weight.t())
            if self.bias is not None:
                output += self.bias
        
        return output
    
    def extra_repr(self):
        return f'in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}'

class ReLU(nn.Module):
    """Функция активации ReLU"""
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return F.relu(x)
    
    def extra_repr(self):
        return ''

class LeakyReLU(nn.Module):
    """Функция активации LeakyReLU"""
    def __init__(self, negative_slope: float = 0.01):
        super().__init__()
        self.negative_slope = negative_slope
    
    def forward(self, x):
        return F.leaky_relu(x, negative_slope=self.negative_slope)
    
    def extra_repr(self):
        return f'negative_slope={self.negative_slope}'

class Sigmoid(nn.Module):
    """Сигмоидальная функция активации"""
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return torch.sigmoid(x)

class Tanh(nn.Module):
    """Гиперболический тангенс"""
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return torch.tanh(x)

class Softmax(nn.Module):
    """Функция Softmax"""
    def __init__(self, dim: int = -1):
        super().__init__()
        self.dim = dim
    
    def forward(self, x):
        return F.softmax(x, dim=self.dim)
    
    def extra_repr(self):
        return f'dim={self.dim}'

# ==================== 2. AUTOENCODER ДЛЯ ШУМОПОДАВЛЕНИЯ ====================

class DenoisingAutoencoder(nn.Module):
    """
    Автоэнкодер для задачи шумоподавления
    """
    def __init__(self, input_dim: int = 784, hidden_dims: List[int] = [512, 256, 128]):
        super().__init__()
        self.input_dim = input_dim
        
        # Энкодер
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(Linear(prev_dim, hidden_dim))
            encoder_layers.append(ReLU())
            prev_dim = hidden_dim
        
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Декодер (симметричная архитектура)
        decoder_layers = []
        hidden_dims_rev = hidden_dims[::-1]
        prev_dim = hidden_dims_rev[0]
        
        for hidden_dim in hidden_dims_rev[1:]:
            decoder_layers.append(Linear(prev_dim, hidden_dim))
            decoder_layers.append(ReLU())
            prev_dim = hidden_dim
        
        # Выходной слой
        decoder_layers.append(Linear(prev_dim, input_dim))
        decoder_layers.append(Sigmoid())
        
        self.decoder = nn.Sequential(*decoder_layers)
    
    def forward(self, x):
        """Прямой проход"""
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed
    
    def add_noise(self, x, noise_level: float = 0.3):
        """Добавление гауссовского шума"""
        noise = torch.randn_like(x) * noise_level
        noisy_x = x + noise
        return torch.clamp(noisy_x, 0.0, 1.0)

# ==================== 3. VARIATIONAL AUTOENCODER ====================

class VariationalAutoencoder(nn.Module):
    """
    Вариационный автоэнкодер
    """
    def __init__(self, input_dim: int = 784, hidden_dim: int = 400, latent_dim: int = 20):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Энкодер
        self.encoder = nn.Sequential(
            Linear(input_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim),
            ReLU()
        )
        
        # Слои для параметров распределения
        self.fc_mu = Linear(hidden_dim, latent_dim)
        self.fc_logvar = Linear(hidden_dim, latent_dim)
        
        # Декодер
        self.decoder = nn.Sequential(
            Linear(latent_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, input_dim),
            Sigmoid()
        )
    
    def encode(self, x):
        """Кодирование в параметры распределения"""
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        """Репараметризация для обратного распространения"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        """Декодирование из латентного пространства"""
        return self.decoder(z)
    
    def forward(self, x):
        """Прямой проход"""
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar
    
    def sample(self, num_samples: int, device: str = 'cpu'):
        """Генерация новых семплов"""
        z = torch.randn(num_samples, self.latent_dim).to(device)
        samples = self.decode(z)
        return samples

def vae_loss(reconstructed, original, mu, logvar, beta: float = 1.0):
    """
    Функция потерь для VAE (ELBO)
    """
    # Reconstruction loss
    recon_loss = F.binary_cross_entropy(reconstructed, original, reduction='mean') * original.size(0)
    
    # KL divergence
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Total loss
    total_loss = recon_loss + beta * kld_loss
    
    return total_loss, recon_loss, kld_loss

# ==================== 4. GENERATIVE ADVERSARIAL NETWORK ====================

class Generator(nn.Module):
    """
    Генератор для GAN
    """
    def __init__(self, latent_dim: int = 100, hidden_dim: int = 256, output_dim: int = 784):
        super().__init__()
        self.latent_dim = latent_dim
        
        self.net = nn.Sequential(
            Linear(latent_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, output_dim),
            Tanh()
        )
    
    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    """
    Дискриминатор для GAN
    """
    def __init__(self, input_dim: int = 784, hidden_dim: int = 256):
        super().__init__()
        
        self.net = nn.Sequential(
            Linear(input_dim, hidden_dim),
            LeakyReLU(0.2),
            Linear(hidden_dim, hidden_dim),
            LeakyReLU(0.2),
            Linear(hidden_dim, hidden_dim),
            LeakyReLU(0.2),
            Linear(hidden_dim, 1),
            Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

class GAN(nn.Module):
    """
    Полная GAN модель
    """
    def __init__(self, latent_dim: int = 100, hidden_dim: int = 256, output_dim: int = 784):
        super().__init__()
        self.generator = Generator(latent_dim, hidden_dim, output_dim)
        self.discriminator = Discriminator(output_dim, hidden_dim)
        self.latent_dim = latent_dim
    
    def generate(self, num_samples: int, device: str = 'cpu'):
        """Генерация новых семплов"""
        z = torch.randn(num_samples, self.latent_dim).to(device)
        return self.generator(z)

# ==================== 5. DENOISING DIFFUSION PROBABILISTIC MODEL ====================

class DiffusionModel(nn.Module):
    """
    Упрощенная DDPM модель
    """
    def __init__(self, input_dim: int = 784, hidden_dim: int = 512, timesteps: int = 100):
        super().__init__()
        self.timesteps = timesteps
        self.input_dim = input_dim
        
        # Параметры диффузионного процесса
        self.register_buffer('betas', self._linear_beta_schedule(timesteps))
        self.register_buffer('alphas', 1. - self.betas)
        self.register_buffer('alphas_cumprod', torch.cumprod(self.alphas, dim=0))
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(self.alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1. - self.alphas_cumprod))
        
        # Нейросеть для предсказания шума
        self.time_embedding = nn.Sequential(
            Linear(1, 16),
            ReLU(),
            Linear(16, 32),
            ReLU(),
            Linear(32, 16)
        )
        
        self.denoiser = nn.Sequential(
            Linear(input_dim + 16, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, input_dim)
        )
    
    def _linear_beta_schedule(self, timesteps, start=0.0001, end=0.02):
        """Линейное расписание beta"""
        return torch.linspace(start, end, timesteps)
    
    def forward(self, x, t):
        """Прямой проход для предсказания шума"""
        # Эмбеддинг временного шага
        t_float = t.float().unsqueeze(-1) / self.timesteps
        t_embed = self.time_embedding(t_float)
        
        # Конкатенация с входными данными
        x_with_t = torch.cat([x, t_embed], dim=-1)
        
        # Предсказание шума
        predicted_noise = self.denoiser(x_with_t)
        return predicted_noise
    
    def q_sample(self, x_start, t, noise=None):
        """Прямой диффузионный процесс (добавление шума)"""
        if noise is None:
            noise = torch.randn_like(x_start)
        
        sqrt_alpha_cumprod_t = self.sqrt_alphas_cumprod[t].view(-1, 1)
        sqrt_one_minus_alpha_cumprod_t = self.sqrt_one_minus_alphas_cumprod[t].view(-1, 1)
        
        return sqrt_alpha_cumprod_t * x_start + sqrt_one_minus_alpha_cumprod_t * noise
    
    def p_sample(self, x, t):
        """Обратный диффузионный процесс (удаление шума)"""
        with torch.no_grad():
            predicted_noise = self.forward(x, t)
            
            alpha_t = self.alphas[t].view(-1, 1)
            sqrt_one_minus_alpha_cumprod_t = self.sqrt_one_minus_alphas_cumprod[t].view(-1, 1)
            
            # Формула обратного процесса
            x_prev = (1 / torch.sqrt(alpha_t)) * (
                x - (self.betas[t].view(-1, 1) / sqrt_one_minus_alpha_cumprod_t) * predicted_noise
            )
            
            if t[0] > 0:
                noise = torch.randn_like(x)
                x_prev += torch.sqrt(self.betas[t].view(-1, 1)) * noise
            
            return x_prev
    
    def generate(self, num_samples, img_shape, device='cpu'):
        """Генерация новых изображений"""
        # Начинаем с чистого шума
        x = torch.randn(num_samples, *img_shape).to(device)
        
        # Обратный процесс
        for t in tqdm(reversed(range(self.timesteps)), desc="Sampling"):
            t_batch = torch.full((num_samples,), t, device=device, dtype=torch.long)
            x = self.p_sample(x, t_batch)
        
        return torch.clamp(x, -1.0, 1.0)

# ==================== 6. AUTOREGRESSIVE MODEL ====================

class MaskedLinear(Linear):
    """
    Маскированный линейный слой для автогрессивных моделей
    """
    def __init__(self, in_features, out_features, mask_type='A', bias=True):
        super().__init__(in_features, out_features, bias)
        
        # Создание маски
        mask = torch.zeros(out_features, in_features)
        
        # Простая маска: разрешаем связи только с предыдущими пикселями
        for i in range(out_features):
            # Разрешаем связи со всеми предыдущими пикселями
            if mask_type == 'A':
                # Маска типа A: запрет связей с текущим и будущими пикселями
                mask[i, :min(i, in_features)] = 1
            else:
                # Маска типа B: разрешение связей с текущим пикселем
                mask[i, :min(i+1, in_features)] = 1
        
        self.register_buffer('mask', mask)
    
    def forward(self, x):
        # Применяем маску к весам перед умножением
        masked_weight = self.weight * self.mask
        return F.linear(x, masked_weight, self.bias)

class AutoregressiveModel(nn.Module):
    """
    Автогрессивная модель (PixelCNN-like)
    """
    def __init__(self, input_dim=784, hidden_dim=256, n_layers=4):
        super().__init__()
        self.input_dim = input_dim
        
        # Упрощенная архитектура
        layers = []
        # Первый слой
        layers.append(MaskedLinear(input_dim, hidden_dim, mask_type='A'))
        layers.append(ReLU())
        
        # Промежуточные слои
        for _ in range(n_layers - 2):
            layers.append(MaskedLinear(hidden_dim, hidden_dim, mask_type='B'))
            layers.append(ReLU())
        
        # Выходной слой
        layers.append(MaskedLinear(hidden_dim, input_dim, mask_type='B'))
        
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        """Предсказание logits для каждого пикселя"""
        return self.net(x)
    
    def generate(self, num_samples, device='cpu'):
        """Последовательная генерация"""
        samples = torch.zeros(num_samples, self.input_dim).to(device)
        
        with torch.no_grad():
            for i in range(self.input_dim):
                if i % 100 == 0:
                    print(f"Generating pixel {i}/{self.input_dim}")
                
                logits = self.forward(samples)
                probs = torch.sigmoid(logits[:, i:i+1])
                
                # Сэмплирование следующего пикселя
                samples[:, i:i+1] = torch.bernoulli(probs)
        
        return samples

# ==================== 7. УТИЛИТЫ ДЛЯ ОБУЧЕНИЯ И ВИЗУАЛИЗАЦИИ ====================

def train_denoising_ae(model, train_loader, test_loader, epochs=20, lr=1e-3, device='cpu'):
    """Обучение автоэнкодера для шумоподавления"""
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.MSELoss()
    
    train_losses = []
    test_losses = []
    
    for epoch in range(epochs):
        # Обучение
        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)):
            data = data.view(data.size(0), -1).to(device)
            
            # Добавление шума
            noisy_data = model.add_noise(data, noise_level=0.3)
            
            # Прямой проход
            reconstructed = model(noisy_data)
            
            # Вычисление лосса
            loss = criterion(reconstructed, data)
            
            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            
            # Градиентный клиппинг для стабильности
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Валидация
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for data, _ in test_loader:
                data = data.view(data.size(0), -1).to(device)
                noisy_data = model.add_noise(data, noise_level=0.3)
                reconstructed = model(noisy_data)
                loss = criterion(reconstructed, data)
                test_loss += loss.item()
        
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        
        print(f'Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Test Loss = {test_loss:.6f}')
    
    return train_losses, test_losses

def train_vae(model, train_loader, test_loader, epochs=30, lr=1e-3, device='cpu', beta=0.001):
    """Обучение VAE"""
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    
    train_losses = []
    recon_losses = []
    kld_losses = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_recon = 0
        total_kld = 0
        
        for batch_idx, (data, _) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)):
            data = data.view(data.size(0), -1).to(device)
            
            # Прямой проход
            reconstructed, mu, logvar = model(data)
            
            # Вычисление лосса
            loss, recon_loss, kld_loss = vae_loss(reconstructed, data, mu, logvar, beta)
            
            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
            total_recon += recon_loss.item()
            total_kld += kld_loss.item()
        
        avg_loss = total_loss / len(train_loader.dataset)
        avg_recon = total_recon / len(train_loader.dataset)
        avg_kld = total_kld / len(train_loader.dataset)
        
        train_losses.append(avg_loss)
        recon_losses.append(avg_recon)
        kld_losses.append(avg_kld)
        
        print(f'Epoch {epoch+1}: Loss = {avg_loss:.6f}, Recon = {avg_recon:.6f}, KLD = {avg_kld:.6f}')
    
    return train_losses, recon_losses, kld_losses

def train_gan(gan_model, train_loader, epochs=50, lr=1e-4, device='cpu'):
    """Обучение GAN"""
    generator = gan_model.generator.to(device)
    discriminator = gan_model.discriminator.to(device)
    
    g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    criterion = nn.BCELoss()
    
    g_losses = []
    d_losses = []
    
    for epoch in range(epochs):
        g_loss_epoch = 0
        d_loss_epoch = 0
        
        for batch_idx, (real_data, _) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)):
            batch_size = real_data.size(0)
            real_data = real_data.view(batch_size, -1).to(device)
            
            # Нормализация в диапазон [-1, 1]
            real_data = (real_data - 0.5) * 2
            
            # Метки для дискриминатора (с шумом для стабильности)
            real_labels = torch.ones(batch_size, 1).to(device) * 0.9  # Label smoothing
            fake_labels = torch.zeros(batch_size, 1).to(device) + 0.1  # Label smoothing
            
            # ========== Обучение дискриминатора ==========
            d_optimizer.zero_grad()
            
            # Реальные данные
            real_output = discriminator(real_data)
            d_real_loss = criterion(real_output, real_labels)
            
            # Сгенерированные данные
            z = torch.randn(batch_size, gan_model.latent_dim).to(device)
            fake_data = generator(z)
            fake_output = discriminator(fake_data.detach())
            d_fake_loss = criterion(fake_output, fake_labels)
            
            # Общий лосс дискриминатора
            d_loss = (d_real_loss + d_fake_loss) / 2
            d_loss.backward()
            torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=1.0)
            d_optimizer.step()
            
            # ========== Обучение генератора ==========
            g_optimizer.zero_grad()
            
            z = torch.randn(batch_size, gan_model.latent_dim).to(device)
            fake_data = generator(z)
            fake_output = discriminator(fake_data)
            
            # Генератор хочет, чтобы дискриминатор считал фейковые данные настоящими
            g_loss = criterion(fake_output, real_labels)
            g_loss.backward()
            torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)
            g_optimizer.step()
            
            g_loss_epoch += g_loss.item()
            d_loss_epoch += d_loss.item()
        
        g_losses.append(g_loss_epoch / len(train_loader))
        d_losses.append(d_loss_epoch / len(train_loader))
        
        print(f'Epoch {epoch+1}: G Loss = {g_losses[-1]:.6f}, D Loss = {d_losses[-1]:.6f}')
        
        # Сохранение лучшей модели GAN
        if epoch > 20 and g_losses[-1] < min(g_losses[:-1]):
            torch.save(gan_model.state_dict(), 'gan_best_model.pth')
            print(f"Saved best GAN model at epoch {epoch+1}")
    
    return g_losses, d_losses

def train_diffusion(model, train_loader, epochs=30, lr=1e-4, device='cpu'):
    """Обучение диффузионной модели"""
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    
    losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for batch_idx, (data, _) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)):
            data = data.view(data.size(0), -1).to(device)
            
            # Нормализация в диапазон [-1, 1]
            data = (data - 0.5) * 2
            
            batch_size = data.size(0)
            
            # Выбор случайного временного шага
            t = torch.randint(0, model.timesteps, (batch_size,), device=device).long()
            
            # Генерация шума
            noise = torch.randn_like(data)
            
            # Добавление шума (прямой процесс)
            noisy_data = model.q_sample(data, t, noise)
            
            # Предсказание шума
            predicted_noise = model(noisy_data, t)
            
            # Вычисление лосса
            loss = F.mse_loss(predicted_noise, noise)
            
            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        
        print(f'Epoch {epoch+1}: Loss = {avg_loss:.6f}')
        
        # Сохранение лучшей модели Diffusion
        if epoch > 10 and losses[-1] < min(losses[:-1]):
            torch.save(model.state_dict(), 'diffusion_best_model.pth')
            print(f"Saved best Diffusion model at epoch {epoch+1}")
    
    return losses

def train_autoregressive(model, train_loader, epochs=20, lr=1e-3, device='cpu'):
    """Обучение автогрессивной модели"""
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.BCEWithLogitsLoss()
    
    losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for batch_idx, (data, _) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)):
            data = data.view(data.size(0), -1).to(device)
            
            # Прямой проход
            logits = model(data)
            
            # Вычисление лосса (бинарная классификация для каждого пикселя)
            loss = criterion(logits, data)
            
            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        
        print(f'Epoch {epoch+1}: Loss = {avg_loss:.6f}')
        
        # Сохранение лучшей модели Autoregressive
        if epoch > 5 and losses[-1] < min(losses[:-1]):
            torch.save(model.state_dict(), 'ar_best_model.pth')
            print(f"Saved best Autoregressive model at epoch {epoch+1}")
    
    return losses

def visualize_results(model, test_loader, model_type='ae', device='cpu', gan_model=None):
    """Визуализация результатов для разных моделей"""
    model.eval()
    
    # Получение тестовых данных
    data, _ = next(iter(test_loader))
    data = data.view(data.size(0), -1).to(device)
    
    fig = plt.figure(figsize=(15, 5))
    
    if model_type == 'ae':
        # Для автоэнкодера: шум -> восстановление
        noisy_data = model.add_noise(data[:10], noise_level=0.3)
        reconstructed = model(noisy_data)
        
        for i in range(10):
            # Original
            ax = fig.add_subplot(3, 10, i+1)
            ax.imshow(data[i].cpu().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Original')
            
            # Noisy
            ax = fig.add_subplot(3, 10, i+11)
            ax.imshow(noisy_data[i].cpu().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Noisy')
            
            # Reconstructed
            ax = fig.add_subplot(3, 10, i+21)
            ax.imshow(reconstructed[i].cpu().detach().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Reconstructed')
    
    elif model_type == 'vae':
        # Для VAE: реконструкция и генерация
        reconstructed, mu, logvar = model(data[:10])
        samples = model.sample(10, device)
        
        for i in range(10):
            # Original
            ax = fig.add_subplot(3, 10, i+1)
            ax.imshow(data[i].cpu().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Original')
            
            # Reconstructed
            ax = fig.add_subplot(3, 10, i+11)
            ax.imshow(reconstructed[i].cpu().detach().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Reconstructed')
            
            # Generated
            ax = fig.add_subplot(3, 10, i+21)
            ax.imshow(samples[i].cpu().detach().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Generated')
    
    elif model_type == 'gan':
        # Для GAN:
        if gan_model is None:
            gan_model = model
        samples = gan_model.generate(10, device)
        samples = (samples + 1) / 2  # Денормализация из [-1, 1] в [0, 1]
        
        for i in range(10):
            ax = fig.add_subplot(1, 10, i+1)
            ax.imshow(samples[i].cpu().detach().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Generated')
    
    elif model_type == 'diffusion':
        # Для диффузионной модели: генерация
        samples = model.generate(10, (784,), device)
        samples = (samples + 1) / 2  # Денормализация
        
        for i in range(10):
            ax = fig.add_subplot(1, 10, i+1)
            ax.imshow(samples[i].cpu().detach().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Generated')
    
    elif model_type == 'ar':
        # Для автогрессивной модели: генерация
        print("Generating autoregressive samples...")
        samples = model.generate(10, device)
        
        for i in range(10):
            ax = fig.add_subplot(1, 10, i+1)
            ax.imshow(samples[i].cpu().detach().view(28, 28), cmap='gray')
            ax.axis('off')
            if i == 0:
                ax.set_title('Generated')
    
    plt.tight_layout()
    plt.show()

# ==================== 8. ОСНОВНОЙ БЛОК ДЛЯ ЗАПУСКА ====================

def main():
    """Основная функция для обучения и оценки всех моделей"""
    print("=" * 80)
    print("КУРСОВАЯ РАБОТА: ГЕНЕРАТИВНЫЕ МОДЕЛИ (УВЕЛИЧЕННОЕ КОЛИЧЕСТВО ЭПОХ)")
    print("=" * 80)
    
    # Проверка доступности GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используемое устройство: {device}")
    
    # Загрузка данных (MNIST)
    print("\nЗагрузка данных MNIST...")
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)  # Увеличили batch size
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print(f"Размер обучающей выборки: {len(train_dataset)}")
    print(f"Размер тестовой выборки: {len(test_dataset)}")
    
    # ========== 1. AUTOENCODER ==========
    print("\n" + "=" * 80)
    print("1. ОБУЧЕНИЕ AUTOENCODER ДЛЯ ШУМОПОДАВЛЕНИЯ (20 эпох)")
    print("=" * 80)
    
    ae_model = DenoisingAutoencoder(input_dim=784, hidden_dims=[512, 256, 128])
    print(f"Параметры модели: {sum(p.numel() for p in ae_model.parameters()):,}")
    
    ae_train_losses, ae_test_losses = train_denoising_ae(
        ae_model, train_loader, test_loader, epochs=20, lr=1e-3, device=device
    )
    
    visualize_results(ae_model, test_loader, model_type='ae', device=device)
    
    # ========== 2. VARIATIONAL AUTOENCODER ==========
    print("\n" + "=" * 80)
    print("2. ОБУЧЕНИЕ VARIATIONAL AUTOENCODER (30 эпох)")
    print("=" * 80)
    
    vae_model = VariationalAutoencoder(input_dim=784, hidden_dim=400, latent_dim=20)
    print(f"Параметры модели: {sum(p.numel() for p in vae_model.parameters()):,}")
    
    vae_losses, recon_losses, kld_losses = train_vae(
        vae_model, train_loader, test_loader, epochs=30, lr=1e-3, device=device, beta=0.001
    )
    
    visualize_results(vae_model, test_loader, model_type='vae', device=device)
    
    # ========== 3. GENERATIVE ADVERSARIAL NETWORK ==========
    print("\n" + "=" * 80)
    print("3. ОБУЧЕНИЕ GENERATIVE ADVERSARIAL NETWORK (50 эпох)")
    print("=" * 80)
    
    gan_model = GAN(latent_dim=100, hidden_dim=256, output_dim=784)
    print(f"Параметры генератора: {sum(p.numel() for p in gan_model.generator.parameters()):,}")
    print(f"Параметры дискриминатора: {sum(p.numel() for p in gan_model.discriminator.parameters()):,}")
    
    g_losses, d_losses = train_gan(
        gan_model, train_loader, epochs=50, lr=1e-4, device=device
    )
    
    visualize_results(gan_model, test_loader, model_type='gan', device=device)
    
    # ========== 4. DIFFUSION MODEL ==========
    print("\n" + "=" * 80)
    print("4. ОБУЧЕНИЕ DIFFUSION MODEL (30 эпох)")
    print("=" * 80)
    
    diffusion_model = DiffusionModel(input_dim=784, hidden_dim=512, timesteps=50)
    print(f"Параметры модели: {sum(p.numel() for p in diffusion_model.parameters()):,}")
    
    diffusion_losses = train_diffusion(
        diffusion_model, train_loader, epochs=30, lr=1e-4, device=device
    )
    
    visualize_results(diffusion_model, test_loader, model_type='diffusion', device=device)
    
    # ========== 5. AUTOREGRESSIVE MODEL ==========
    print("\n" + "=" * 80)
    print("5. ОБУЧЕНИЕ AUTOREGRESSIVE MODEL (20 эпох)")
    print("=" * 80)
    
    ar_model = AutoregressiveModel(input_dim=784, hidden_dim=256, n_layers=4)
    print(f"Параметры модели: {sum(p.numel() for p in ar_model.parameters()):,}")
    
    ar_losses = train_autoregressive(
        ar_model, train_loader, epochs=20, lr=1e-3, device=device
    )
    
    visualize_results(ar_model, test_loader, model_type='ar', device=device)
    
    # ========== ВИЗУАЛИЗАЦИЯ КРИВЫХ ОБУЧЕНИЯ ==========
    print("\n" + "=" * 80)
    print("ВИЗУАЛИЗАЦИЯ КРИВЫХ ОБУЧЕНИЯ ВСЕХ МОДЕЛЕЙ")
    print("=" * 80)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # AE
    axes[0, 0].plot(ae_train_losses, label='Train', linewidth=2)
    axes[0, 0].plot(ae_test_losses, label='Test', linewidth=2)
    axes[0, 0].set_title('Autoencoder (MSE Loss)', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].text(0.7, 0.9, f'Final: {ae_test_losses[-1]:.6f}', 
                   transform=axes[0, 0].transAxes, fontsize=10)
    
    # VAE
    axes[0, 1].plot(vae_losses, label='Total Loss', linewidth=2)
    axes[0, 1].plot(recon_losses, label='Reconstruction', linewidth=2)
    axes[0, 1].plot(kld_losses, label='KLD', linewidth=2)
    axes[0, 1].set_title('Variational Autoencoder', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # GAN
    axes[0, 2].plot(g_losses, label='Generator', linewidth=2)
    axes[0, 2].plot(d_losses, label='Discriminator', linewidth=2)
    axes[0, 2].set_title('GAN Losses', fontsize=12, fontweight='bold')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Loss')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    axes[0, 2].text(0.7, 0.9, f'Final G: {g_losses[-1]:.4f}\nFinal D: {d_losses[-1]:.4f}', 
                   transform=axes[0, 2].transAxes, fontsize=9)
    
    # Diffusion
    axes[1, 0].plot(diffusion_losses, linewidth=2, color='green')
    axes[1, 0].set_title('Diffusion Model (MSE Loss)', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].text(0.7, 0.9, f'Final: {diffusion_losses[-1]:.4f}', 
                   transform=axes[1, 0].transAxes, fontsize=10)
    
    # AR
    axes[1, 1].plot(ar_losses, linewidth=2, color='purple')
    axes[1, 1].set_title('Autoregressive Model (BCE Loss)', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].text(0.7, 0.9, f'Final: {ar_losses[-1]:.4f}', 
                   transform=axes[1, 1].transAxes, fontsize=10)
    
    # Сводная таблица
    axes[1, 2].axis('off')
    
    # Расчет улучшений
    ae_improvement = ((ae_train_losses[0] - ae_train_losses[-1]) / ae_train_losses[0] * 100) if ae_train_losses[0] > 0 else 0
    vae_improvement = ((vae_losses[0] - vae_losses[-1]) / vae_losses[0] * 100) if vae_losses[0] > 0 else 0
    diff_improvement = ((diffusion_losses[0] - diffusion_losses[-1]) / diffusion_losses[0] * 100) if diffusion_losses[0] > 0 else 0
    ar_improvement = ((ar_losses[0] - ar_losses[-1]) / ar_losses[0] * 100) if ar_losses[0] > 0 else 0
    
    summary_text = (
        "ИТОГОВАЯ СВОДКА ПОСЛЕ УВЕЛИЧЕНИЯ ЭПОХ:\n\n"
        f"Autoencoder (20 эпох):\n"
        f"  Train Loss: {ae_train_losses[-1]:.6f}\n"
        f"  Test Loss: {ae_test_losses[-1]:.6f}\n"
        f"  Улучшение: {ae_improvement:.1f}%\n\n"
        f"VAE (30 эпох):\n"
        f"  Total Loss: {vae_losses[-1]:.6f}\n"
        f"  Recon Loss: {recon_losses[-1]:.6f}\n"
        f"  KLD: {kld_losses[-1]:.6f}\n"
        f"  Улучшение: {vae_improvement:.1f}%\n\n"
        f"GAN (50 эпох):\n"
        f"  G Loss: {g_losses[-1]:.4f}\n"
        f"  D Loss: {d_losses[-1]:.4f}\n\n"
        f"Diffusion (30 эпох):\n"
        f"  Loss: {diffusion_losses[-1]:.4f}\n"
        f"  Улучшение: {diff_improvement:.1f}%\n\n"
        f"Autoregressive (20 эпох):\n"
        f"  Loss: {ar_losses[-1]:.4f}\n"
        f"  Улучшение: {ar_improvement:.1f}%"
    )
    axes[1, 2].text(0.05, 0.5, summary_text, fontsize=9, 
                   verticalalignment='center', fontfamily='monospace',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.suptitle('КРИВЫЕ ОБУЧЕНИЯ ГЕНЕРАТИВНЫХ МОДЕЛЕЙ (УВЕЛИЧЕННОЕ КОЛИЧЕСТВО ЭПОХ)', 
                fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    print("\n" + "=" * 80)
    print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
    print("=" * 80)
    
    # Сохранение моделей
    print("\nСохранение моделей...")
    models_dict = {
        'ae': ae_model,
        'vae': vae_model,
        'gan': gan_model,
        'diffusion': diffusion_model,
        'ar': ar_model
    }
    
    for name, model in models_dict.items():
        torch.save(model.state_dict(), f'{name}_model_extended.pth')
        print(f"Модель {name} сохранена в {name}_model_extended.pth")
    
    # Анализ результатов
    print("\n" + "=" * 80)
    print("АНАЛИЗ РЕЗУЛЬТАТОВ")
    print("=" * 80)
    
    print(f"\nAutoencoder:")
    print(f"  Начальный loss: {ae_train_losses[0]:.6f}")
    print(f"  Финальный loss: {ae_train_losses[-1]:.6f}")
    print(f"  Улучшение: {ae_improvement:.1f}%")
    
    print(f"\nVariational Autoencoder:")
    print(f"  Reconstruction улучшился с {recon_losses[0]:.6f} до {recon_losses[-1]:.6f}")
    print(f"  KLD увеличился с {kld_losses[0]:.6f} до {kld_losses[-1]:.6f}")
    
    print(f"\nGAN:")
    print(f"  Минимальный Generator Loss: {min(g_losses):.4f} (эпоха {g_losses.index(min(g_losses))+1})")
    print(f"  Минимальный Discriminator Loss: {min(d_losses):.4f} (эпоха {d_losses.index(min(d_losses))+1})")
    
    print(f"\nDiffusion Model:")
    print(f"  Начальный loss: {diffusion_losses[0]:.4f}")
    print(f"  Финальный loss: {diffusion_losses[-1]:.4f}")
    print(f"  Улучшение: {diff_improvement:.1f}%")
    
    print(f"\nAutoregressive Model:")
    print(f"  Начальный loss: {ar_losses[0]:.4f}")
    print(f"  Финальный loss: {ar_losses[-1]:.4f}")
    print(f"  Улучшение: {ar_improvement:.1f}%")
    
    return models_dict

if __name__ == "__main__":
    # Запуск основной функции
    trained_models = main()